In [21]:
import pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [22]:
article_df = pd.read_csv("datas/article.csv")
print(len(article_df))
article_df.tail(1)

240


,title,link,category,content
239,"중기 경기전망지수 내리막...8월 79.0, 전년 比 3.0%↓",https://news.naver.com/main/read.nhn?mode=LSD&...,105,"중소기업 경기전망이 지난 2월 역대 최저점(76.3)을 나타낸 이후, 올해 두번째로..."


In [23]:
train_x, test_x, train_y, test_y = train_test_split(article_df.content, article_df.category, test_size=0.3, random_state=1)
train_x.shape, test_x.shape, train_y.shape, test_y.shape

((168,), (72,), (168,), (72,))

In [24]:
clf=Pipeline([
    ('vect', TfidfVectorizer()),
    ("clf", MultinomialNB(alpha=0.01)),
])

In [25]:
model=clf.fit(train_x, train_y)

In [26]:
pred_y = model.predict(test_x)

In [27]:
len(pred_y[pred_y == test_y.values])/len(pred_y)

0.7361111111111112

In [28]:
print(classification_report(test_y, pred_y))

              precision    recall  f1-score   support

         100       0.74      0.93      0.82        15
         101       0.58      0.70      0.64        10
         102       0.79      0.73      0.76        15
         103       0.67      0.67      0.67         6
         104       0.77      0.77      0.77        13
         105       0.88      0.54      0.67        13

    accuracy                           0.74        72
   macro avg       0.74      0.72      0.72        72
weighted avg       0.75      0.74      0.73        72



In [29]:
with open('article/models/model.pkl', 'wb') as file:
    pickle.dump(model, file)

In [30]:
with open('article/models/model.pkl','rb') as file:
    load_model = pickle.load(file)
load_model

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clf', MultinomialNB(alpha=0.01))])

In [31]:
categories ={
    100:'정치',101:'경제',102:'사회',103:'생활문화', 104:'세계', 105:'IT/과학'
}

In [32]:
text='요즘 환율과 주가 데이터 예측 불가능'
category =load_model.predict([text])[0]
categories[category]

'경제'

In [33]:
proba =np.round(load_model.predict_proba([text])*100,2)
list(zip(categories.values(), proba[0]))

[('정치', 6.94),
 ('경제', 46.52),
 ('사회', 9.94),
 ('생활문화', 19.4),
 ('세계', 7.55),
 ('IT/과학', 9.66)]